In [2]:
import os
import sys
import logging
import numpy as np
import pandas as pd
import urlparse
os.sys.path.append("/home/sparrow/sparrow/malicious_url_cluster")
from config import cfg
from logger import logger

cfg.N_CLUSTER_RATION = 200

cfg.EDIT_DISTANCE_THRESH_LONG = 0.3
cfg.EDIT_DISTANCE_THRESH_SHORT = 0.3
cfg.LONG_URL_THRESH = 100
cfg.SHORT_URL_THRESH = 4

cfg.MIN_SUBSTRING_SIZE = 4
cfg.SINGLE_REGEX_SIZE = 4
cfg.TOTAL_REGEX_SIZE = 4
cfg.SINGLE_REGEX_SIZE_RATIO = 0.2
cfg.TOTAL_REGEX_SIZE_RATIO = 0.2

cfg.DOMAIN_CLUSTER_SIZE_THRESH = 3

import preprocess
import vectorize
import clustering
import regex_url
import urlnormalize
import regex_domain
from file_io import load_urls, dump_urls
import url_forest
reload(url_forest)
import joblib as jl

## Prepare Testing Set

In [7]:
def get_malware_edr_data(file_list):
    root_path = "/home/sparrow/sparrow/data/sangfor/18"
    df_list = []
    for i in file_list:
        local_path = root_path + i
        filepath = os.path.join(local_path ,'dumped_result.edr.csv')
        df_list.append( pd.read_csv(filepath))
        print local_path
        
    df = pd.concat(df_list, axis = 0)
    return df

file_list = ["0401", "0402", "0410", "0411", "0412", "0415", "0416", "0417", "0418", "0419"]
df = get_malware_edr_data(file_list)
df = df.drop_duplicates("url")
urls = list(df.url)
print "\nEDR total count: %d\n" % len(df)

url_0 = list(df.loc[df.attr == 0].url)
url_1 = list(df.loc[df.attr == 1].url)
url_2 = list(df.loc[df.attr == 2].url)
url_3 = list(df.loc[df.attr == 3].url)
print "0 malware\t%d\n1 benign\t%d\n2 unknown\t%d\n3 possible benign\t%d" %(len(url_0), len(url_1), len(url_2), len(url_3))
df = df.set_index("url")

/home/sparrow/sparrow/data/sangfor/180401
/home/sparrow/sparrow/data/sangfor/180402
/home/sparrow/sparrow/data/sangfor/180410
/home/sparrow/sparrow/data/sangfor/180411
/home/sparrow/sparrow/data/sangfor/180412
/home/sparrow/sparrow/data/sangfor/180415
/home/sparrow/sparrow/data/sangfor/180416
/home/sparrow/sparrow/data/sangfor/180417
/home/sparrow/sparrow/data/sangfor/180418
/home/sparrow/sparrow/data/sangfor/180419

EDR total count: 36073

0 malware	2801
1 benign	296
2 unknown	32972
3 possible benign	4


## Prepare Trainning Data

In [29]:
sangfor_url = list(set(load_urls("../../data/sangfor/malware_all_0419.csv", csv = True)))
vt_url_su = list(set(load_urls("../../data/VirusTotalnew/malicious/180421-ma.txt", csv = False)))
vt_url_ma = list(set(load_urls("../../data/VirusTotalnew/malicious/180421-su.txt", csv = False)))
train_urls = list(set(sangfor_url + vt_url_su + vt_url_ma) )
print len(train_urls)

/home/sparrow/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2818: DtypeWarning: Columns (6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
2018-04-25 14:11:52 file_io.py [line:52] DEBUG	URLs Count:	71201
2018-04-25 14:11:52 file_io.py [line:52] DEBUG	URLs Count:	104525
2018-04-25 14:11:53 file_io.py [line:52] DEBUG	URLs Count:	850584


994836


In [31]:
994836-39789

955047

In [20]:
dump_urls(train_urls, "../../data/EXP_domain/malware.csv")

2018-04-21 11:25:39 file_io.py [line:33] DEBUG	URLs has been dump	../../data/EXP_domain/malware.csv


In [4]:
url_domain, url_path = preprocess.make_url_split(train_urls)
print len(url_domain)
print len(url_path)
urls = list(url_domain)

360770
519300


In [13]:
cc_list = clustering.make_kmeans_cluster_mass(urls,domain = True, path = False, param = False)

2018-04-21 10:58:05 vectorize.py [line:77] DEBUG	vectorization complete! data shape:	(360770, 128)
2018-04-21 10:58:05 clustering.py [line:224] DEBUG	Begin to make k-means cluster, data_size: 360770 n_clusters: 180
2018-04-21 10:58:05 clustering.py [line:245] DEBUG	optimal n_cluster	7
2018-04-21 10:58:40 clustering.py [line:263] DEBUG	k-means cluster done!
2018-04-21 10:58:40 clustering.py [line:291] DEBUG	Preliminary K-means clustering complete
2018-04-21 10:58:44 vectorize.py [line:77] DEBUG	vectorization complete! data shape:	(50867, 128)
2018-04-21 10:58:44 clustering.py [line:224] DEBUG	Begin to make k-means cluster, data_size: 50867 n_clusters: 25
2018-04-21 10:58:44 clustering.py [line:245] DEBUG	optimal n_cluster	3
2018-04-21 10:58:50 clustering.py [line:263] DEBUG	k-means cluster done!
2018-04-21 10:58:56 vectorize.py [line:77] DEBUG	vectorization complete! data shape:	(98049, 128)
2018-04-21 10:58:56 clustering.py [line:224] DEBUG	Begin to make k-means cluster, data_size: 980

In [14]:
clustering.make_string_distance_cluster(data = cc_list, 
                            metric = "distance",  
                            file_path = "../../data/EXP_domain/cluster_distance.json", )

index 0 0/41535
index 0 0/35683
index 0 0/35620
index 0 0/30939
index 0 0/26746
index 0 0/26739
index 0 0/25817
index 0 0/19861
index 100 100/19861
index 100 100/30939
index 100 100/26746
index 100 100/35620
index 100 100/35683
index 200 200/19861
index 200 200/30939
index 100 100/25817
index 300 300/30939
index 200 200/26746
index 400 400/19861
index 200 200/35620
index 200 200/35683
index 100 100/26739
index 200 200/25817
index 900 900/41535
index 300 300/26746
index 500 500/30939
index 600 600/19861
index 300 300/35620
index 700 700/19861
index 600 600/30939
index 300 300/25817
index 1300 1300/41535
index 400 400/26746
index 800 800/19861
index 700 700/30939
index 900 900/19861
index 400 400/35620
index 1600 1600/41535
index 200 200/26739
index 400 400/35683
index 800 800/30939
index 500 500/26746
index 1000 1000/19861
index 400 400/25817
index 900 900/30939
index 1200 1200/19861
index 500 500/35620
index 500 500/35683
index 600 600/26746
index 1000 1000/30939
index 1300 1300/19861


index 39800 39800/41535
index 39900 39900/41535
index 3900 3900/35683
index 40800 40800/41535
index 41000 41000/41535
index 41200 41200/41535
index 41400 41400/41535
index 9400 9400/30939
index 0 0/19303
index 1500 1500/19656
index 3900 3900/35620
index 5000 5000/26746
index 9500 9500/30939
index 100 100/19303
index 1600 1600/19656
index 2200 2200/26739
index 4000 4000/35683
index 5100 5100/26746
index 200 200/19303
index 4000 4000/35620
index 9900 9900/30939
index 4900 4900/25817
index 5200 5200/26746
index 4100 4100/35683
index 10000 10000/30939
index 400 400/19303
index 10100 10100/30939
index 2000 2000/19656
index 10200 10200/30939
index 5300 5300/26746
index 2100 2100/19656
index 10300 10300/30939
index 4200 4200/35683
index 5100 5100/25817
index 2200 2200/19656
index 600 600/19303
index 10400 10400/30939
index 5400 5400/26746
index 4200 4200/35620
index 10500 10500/30939
index 700 700/19303
index 10600 10600/30939
index 5500 5500/26746
index 4300 4300/35683
index 800 800/19303
in

index 27200 27200/30939
index 15600 15600/19303
index 27300 27300/30939
index 27400 27400/30939
index 15800 15800/19303
index 27500 27500/30939
index 27600 27600/30939
index 27700 27700/30939
index 16100 16100/19303
index 27800 27800/30939
index 7200 7200/35620
index 27900 27900/30939
index 16300 16300/19303
index 28000 28000/30939
index 10400 10400/25817
index 16400 16400/19303
index 28100 28100/30939
index 28200 28200/30939
index 9900 9900/26746
index 1300 1300/17140
index 16600 16600/19303
index 16700 16700/19303
index 28400 28400/30939
index 28600 28600/30939
index 17000 17000/19303
index 28700 28700/30939
index 28800 28800/30939
index 28900 28900/30939
index 10500 10500/25817
index 29100 29100/30939
index 29400 29400/30939
index 18000 18000/19303
index 18100 18100/19303
index 29700 29700/30939
index 29800 29800/30939
index 29900 29900/30939
index 30000 30000/30939
index 18700 18700/19303
index 30300 30300/30939
index 30400 30400/30939
index 30500 30500/30939
index 30600 30600/3093

index 19300 19300/25817
index 2700 2700/11547
index 5900 5900/14661
index 9900 9900/17140
index 19400 19400/25817
index 15400 15400/26746
index 10000 10000/17140
index 2900 2900/11547
index 10100 10100/17140
index 10400 10400/35683
index 6100 6100/14661
index 19700 19700/25817
index 10300 10300/35620
index 3100 3100/11547
index 6300 6300/26739
index 3200 3200/11547
index 3300 3300/11547
index 6300 6300/14661
index 15700 15700/26746
index 20100 20100/25817
index 10500 10500/35683
index 3500 3500/11547
index 10400 10400/35620
index 20300 20300/25817
index 15800 15800/26746
index 11100 11100/17140
index 3700 3700/11547
index 20500 20500/25817
index 11400 11400/17140
index 3800 3800/11547
index 15900 15900/26746
index 11500 11500/17140
index 3900 3900/11547
index 6400 6400/26739
index 11600 11600/17140
index 20700 20700/25817
index 10500 10500/35620
index 6700 6700/14661
index 16000 16000/26746
index 4100 4100/11547
index 20900 20900/25817
index 21000 21000/25817
index 6800 6800/14661
inde

index 6500 6500/7910
index 6600 6600/7910
index 6800 6800/7910
index 7000 7000/7910
index 13600 13600/35620
index 7400 7400/7910
index 7800 7800/7910
index 7900 7900/7910
index 13800 13800/35683
index 8500 8500/26739
index 13900 13900/35683
index 13800 13800/35620
index 14000 14000/35620
index 14200 14200/35683
index 8800 8800/26739
index 14100 14100/35620
index 14200 14200/35620
index 14400 14400/35683
index 8900 8900/26739
index 14300 14300/35620
index 14500 14500/35683
index 14400 14400/35620
index 9000 9000/26739
index 14500 14500/35620
index 14700 14700/35683
index 14600 14600/35620
index 14800 14800/35683
index 14900 14900/35620
index 15000 15000/35683
index 15000 15000/35620
index 15100 15100/35683
index 9400 9400/26739
index 8800 8800/11329
index 15200 15200/35620
index 15300 15300/35683
index 15300 15300/35620
index 15400 15400/35683
index 9500 9500/26739
index 10000 10000/11329
index 15400 15400/35620
index 15600 15600/35683
index 15600 15600/35620
index 9700 9700/26739
index

index 35100 35100/35620
index 35300 35300/35620
index 35500 35500/35620
index 35600 35600/35620
index 32000 32000/35683
index 32100 32100/35683
index 18300 18300/26739
index 32300 32300/35683
index 32400 32400/35683
index 18400 18400/26739
index 32600 32600/35683
index 32700 32700/35683
index 32800 32800/35683
index 32900 32900/35683
index 33000 33000/35683
index 18500 18500/26739
index 33100 33100/35683
index 33200 33200/35683
index 33300 33300/35683
index 33400 33400/35683
index 33500 33500/35683
index 18600 18600/26739
index 33700 33700/35683
index 33800 33800/35683
index 33900 33900/35683
index 34000 34000/35683
index 34100 34100/35683
index 34300 34300/35683
index 34500 34500/35683
index 34600 34600/35683
index 34700 34700/35683
index 34800 34800/35683
index 34900 34900/35683
index 35000 35000/35683
index 35100 35100/35683
index 35200 35200/35683
index 35300 35300/35683
index 35500 35500/35683
index 18800 18800/26739
index 18900 18900/26739
index 19400 19400/26739
index 19600 1960

2018-04-21 11:16:17 clustering.py [line:109] DEBUG	string distance time cost:	923.536451


In [18]:
regex_list = regex_domain.domain_regex_extract(
    input_file_path = "../../data/EXP_domain/cluster_distance.json",
    output_file_path = "../../data/EXP_domain/regex_raw.txt",
    cluster_size_thresh=6)

2018-04-21 11:23:09 file_io.py [line:172] DEBUG	Cluster Data has been loaded	../../data/EXP_domain/cluster_distance.json
2018-04-21 11:23:09 regex_domain.py [line:319] DEBUG	total cluster num:	185589
2018-04-21 11:23:09 regex_domain.py [line:321] DEBUG	big cluster:	12287
2018-04-21 11:23:09 regex_domain.py [line:323] DEBUG	small cluster:	31742
2018-04-21 11:23:09 regex_domain.py [line:324] DEBUG	single one:	141560
2018-04-21 11:23:09 regex_domain.py [line:325] DEBUG	cluster size detail:	Counter({1: 141560, 2: 20441, 3: 7428, 4: 3873, 5: 2394, 6: 1660, 7: 1222, 8: 956, 9: 758, 10: 621, 11: 539, 12: 415, 13: 385, 14: 301, 15: 234, 16: 234, 17: 202, 18: 202, 19: 149, 21: 118, 20: 113, 22: 96, 23: 81, 25: 70, 24: 66, 26: 53, 31: 51, 27: 47, 28: 46, 30: 44, 33: 39, 29: 37, 32: 33, 34: 32, 35: 28, 39: 28, 38: 27, 37: 25, 36: 24, 45: 23, 50: 22, 41: 21, 40: 18, 44: 18, 57: 17, 42: 16, 51: 16, 43: 15, 47: 15, 52: 15, 61: 15, 70: 14, 46: 13, 58: 13, 63: 13, 66: 13, 67: 13, 74: 13, 64: 12, 56: 1

2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,6}\.nljhh\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'nljhh'], 2: ['\\w{3}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3}\.nljhh\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'nhdny'], 2: ['\\w{3,6}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,6}\.nhdny\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'llhrx'], 2: ['\\w{2,5}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{2,5}\.llhrx\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'pttfy'], 2: ['\\w{3}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3}\.pttfy\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'faith'], 1: [u'vnhu'], 2: [u'99d9n'], 3: ['\\w{

2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{2,5}\.llhrx\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'krwrn'], 2: ['\\w{3,5}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,5}\.krwrn\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'051intn'], 2: [u'q']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^q\.051intn\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'npxvb'], 2: ['\\w{2,5}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{2,5}\.npxvb\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'rygdd'], 2: ['\\w{3}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3}\.rygdd\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'llhrx'], 2: ['\\w{3,5}']})
2018-04-21 1

2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,5}\.npxvb\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'npxvb'], 2: ['\\w{3,5}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,5}\.npxvb\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'krwrn'], 2: ['\\w{3,5}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,5}\.krwrn\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'knyxj'], 2: ['\\w{3}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3}\.knyxj\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'nljhh'], 2: ['\\w{3,6}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,6}\.nljhh\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'97sky'], 2: ['\\w{3,4}']})
2018

2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'nhklg'], 2: ['\\w{3,6}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,6}\.nhklg\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'nhdny'], 2: ['\\w{3,6}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,6}\.nhdny\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'nhdny'], 2: ['\\w{3,6}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,6}\.nhdny\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'nhdny'], 2: ['\\w{3,6}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,6}\.nhdny\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'win'], 1: [u'fnkjf'], 2: [u'ckq02'], 3: ['\\w{6}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{6}\

...

2018-04-21 11:23:32 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'faith'], 1: [u'vjde'], 2: [u'www'], 3: [u'thvgl'], 4: [u'q81huf'], 5: [u'0gdind'], 6: ['\\w{4,6}']})
2018-04-21 11:23:32 regex_domain.py [line:299] DEBUG	^\w{4,6}\.0gdind\.q81huf\.thvgl\.www\.vjde\.faith$
2018-04-21 11:23:32 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'k2jis'], 2: [u'www'], 3: [u'pm2zj'], 4: ['\\w{6}']})
2018-04-21 11:23:32 regex_domain.py [line:299] DEBUG	^\w{6}\.pm2zj\.www\.k2jis\.download$
2018-04-21 11:23:32 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'x4yil'], 2: [u'www'], 3: [u'5dr6f'], 4: ['\\w{4,6}'], 5: ['\\w{5,6}']})
2018-04-21 11:23:32 regex_domain.py [line:299] DEBUG	^\w{5,6}\.\w{4,6}\.5dr6f\.www\.x4yil\.download$
2018-04-21 11:23:32 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'faith'], 1: [u'josid'], 2: [u'iu22q'], 3: [u'21026'], 4: [u'y5fekp'], 5: [u'0snod4', u'huf4rn']

2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^\w{5,6}\.1mab9\.www\.8ibv1\.download$
2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'k2jis'], 2: [u'www'], 3: [u'pm2zj'], 4: [u'85051'], 5: ['\\w{5,6}']})
2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^\w{5,6}\.85051\.pm2zj\.www\.k2jis\.download$
2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'win'], 1: [u'478sk'], 2: [u'www'], 3: [u'6ndldt'], 4: [u'62469'], 5: ['\\w{6}'], 6: ['\\w{6}']})
2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^\w{6}\.\w{6}\.62469\.6ndldt\.www\.478sk\.win$
2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'win'], 1: [u'dichanpinglun'], 2: [u'www'], 3: [u'krq1e'], 4: [u'bmztor'], 5: [u'92862'], 6: [u'44782', u'z4djf8', u'3l8y8x', u'9f4gal']})
2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^(:?44782|z4djf8|3l8y8x|9f4gal)\.92862\.bmztor\.krq1e\.www\.dichanpinglu

2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'2r40f'], 2: [u'www'], 3: [u'26cd5'], 4: [u'9kbb4n', u'53227', u'62505', u'o4ej0p'], 5: [u'o8cd0v', u'26403', u'dzv5lh', u'81994']})
2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^(:?o8cd0v|26403|dzv5lh|81994)\.(:?9kbb4n|53227|62505|o4ej0p)\.26cd5\.www\.2r40f\.download$
2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'ar9r6'], 2: [u'www'], 3: [u'09167'], 4: ['\\w{6}'], 5: ['\\w{5,6}']})
2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^\w{5,6}\.\w{6}\.09167\.www\.ar9r6\.download$
2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'k2jis'], 2: [u'www'], 3: [u'8vjai'], 4: [u'45176', u'4evc8c', u'83680', u'78284'], 5: [u'17640', u'rxtw72', u'14953', u'9tqgfe']})
2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^(:?17640|rxtw72|14953|9tqgfe)\.(:?45176|4evc8c|83680|7

2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'q3537'], 2: [u'www'], 3: [u'dllm3'], 4: [u'o147im'], 5: ['\\w{5,6}']})
2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^\w{5,6}\.o147im\.dllm3\.www\.q3537\.download$
2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'k8pnq'], 2: [u'www'], 3: [u'8wetiv'], 4: [u'cs', u'v3sc9e'], 5: ['\\w{5,6}']})
2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^\w{5,6}\.(:?cs|v3sc9e)\.8wetiv\.www\.k8pnq\.download$
2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'sx'], 2: [u'adsl-pool'], 3: [u'118'], 4: [u'81'], 5: ['\\d{1,3}'], 6: ['\\d{2,3}']})
2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^\d{2,3}\.\d{1,3}\.81\.118\.adsl-pool\.sx\.cn$
2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'c1rdu'], 2: [u'85543'], 3: [u'tl9l5z'], 4: 

2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^[^\.]{62}\.(:?0-0-0-0-0-0-0-0-0-0-0-0-0-7-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-59-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-41-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-45-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-53-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-38-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-33-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-47-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-56-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-60-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-28-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-44-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-23-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-46-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-57-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-19-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-58-0-0-0-0-0-0-0-0-0-0-0-0-

In [2]:
df = regex_domain.domain_regex_check(
    input_file_path          = "../../data/EXP_domain/regex_raw.txt", 
    test_benign_file_path    = "../../data/sangfor/safe_0419.csv",
    test_malicious_file_path = "../../data/EXP_domain/malware.csv",
    result_file_path         = "../../data/EXP_domain/regex_result.txt", 
    n_jobs = 8)

2018-04-21 11:58:14 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_domain/regex_raw.txt
2018-04-21 11:58:16 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/sangfor/safe_0419.csv
2018-04-21 11:58:18 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/EXP_domain/malware.csv
2018-04-21 11:58:18 regex_domain.py [line:408] DEBUG	regex count: 4586
2018-04-21 11:58:18 regex_domain.py [line:409] DEBUG	benign urls count: 1461039
2018-04-21 11:58:18 regex_domain.py [line:410] DEBUG	malicious urls count: 994836


batch index 0	sample index 0	FP(url) 0	FP(domain) 0	TP(url) 9	TP(domain) 8	^\w{4,6}\.tti41k\.64ryqf\.9s40r\.www\.vird\.win$
batch index 0	sample index 1	FP(url) 0	FP(domain) 0	TP(url) 11	TP(domain) 11	^\w{6}\.p9fev\.www\.fgi5\.win$
batch index 0	sample index 2	FP(url) 0	FP(domain) 0	TP(url) 12	TP(domain) 12	^\w{6}\.wuo6c\.f8d9\.faith$
batch index 1	sample index 0	FP(url) 0	FP(domain) 0	TP(url) 38	TP(domain) 30	^\w{5,6}\.8we8w\.fkls\.faith$
batch index 0	sample index 3	FP(url) 0	FP(domain) 0	TP(url) 29	TP(domain) 4	^\w{4,6}\.\w{5,6}\.47160\.ftt\.(:?y3mx35|11xpwb)\.wymigx\.zjwpq1\.kskddh\.co0qvm\.uul0jd\.dllm3\.www\.q3537\.download$
batch index 1	sample index 1	FP(url) 0	FP(domain) 0	TP(url) 14	TP(domain) 14	^\d{2,3}\.(:?003096|003097)\.com$
batch index 0	sample index 4	FP(url) 9247	FP(domain) 3161	TP(url) 249	TP(domain) 98	^www\.[A-Za-z]{4,8}\.com\.cn$
batch index 1	sample index 2	FP(url) 0	FP(domain) 0	TP(url) 14	TP(domain) 14	^\w{5,6}\.37l13\.dsnjfk\.win$
batch index 2	sample index 0	

batch index 6	sample index 2	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^nowinstallupgradefreeforsystemssetpcandmac\.win$
batch index 1	sample index 14	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^yourbestandfinest4upgradingalways\.(:?bid|top)$
batch index 4	sample index 7	FP(url) 0	FP(domain) 0	TP(url) 24	TP(domain) 17	^\w{5,6}\.m4s2o\.jkdf\.faith$
batch index 3	sample index 10	FP(url) 0	FP(domain) 0	TP(url) 28	TP(domain) 24	^\w{5,6}\.o4ccu\.ajio\.win$
batch index 7	sample index 0	FP(url) 0	FP(domain) 0	TP(url) 9	TP(domain) 7	^\w{3,6}\.6219y\.vqlln\.download$
batch index 0	sample index 17	FP(url) 0	FP(domain) 0	TP(url) 24	TP(domain) 22	^[A-Za-z]{5,8}\.en\.world-stone\.com$
batch index 5	sample index 5	FP(url) 0	FP(domain) 0	TP(url) 7	TP(domain) 7	^\w{5,6}\.bhc6h\.www\.vrfd\.faith$
batch index 2	sample index 13	FP(url) 0	FP(domain) 0	TP(url) 30	TP(domain) 28	^\w{4,6}\.t8eon\.mu3y4\.download$
batch index 6	sample index 3	FP(url) 0	FP(domain) 0	TP(url) 23	TP(domain) 19	^\w{5,6}\.p77

batch index 2	sample index 21	FP(url) 0	FP(domain) 0	TP(url) 21	TP(domain) 4	^\w{5,6}\.47160\.(:?www|ftt)\.11xpwb\.wymigx\.zjwpq1\.kskddh\.co0qvm\.uul0jd\.dllm3\.www\.q3537\.download$
batch index 4	sample index 16	FP(url) 0	FP(domain) 0	TP(url) 12	TP(domain) 11	^\w{5,6}\.d6793\.www\.itor\.win$
batch index 6	sample index 11	FP(url) 1	FP(domain) 1	TP(url) 19	TP(domain) 6	^down\w{0,6}\.52zsoft\.com$
batch index 1	sample index 23	FP(url) 0	FP(domain) 0	TP(url) 12	TP(domain) 10	^\w{5,6}\.7j88p\.www\.fjvru8\.win$
batch index 3	sample index 19	FP(url) 0	FP(domain) 0	TP(url) 16	TP(domain) 15	^\w{5,6}\.3rb73\.fiei3\.win$
batch index 0	sample index 26	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^ask24updateultimateformacandpc\.(:?info|win|top)$
batch index 5	sample index 14	FP(url) 0	FP(domain) 0	TP(url) 7	TP(domain) 7	^\d{5}\.vdj15\.vnhu\.faith$
batch index 7	sample index 9	FP(url) 0	FP(domain) 0	TP(url) 13	TP(domain) 12	^\w{5,6}\.www\.478sk\.win$
batch index 2	sample index 22	FP(url) 0	FP(do

batch index 1	sample index 31	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^(:?36126|26050)\.(:?7h03er|2n344q)\.fwc56\.www\.gjoie\.faith$
batch index 0	sample index 34	FP(url) 0	FP(domain) 0	TP(url) 28	TP(domain) 18	^\w{4,6}\.xvft3\.fnjks\.win$
batch index 2	sample index 30	FP(url) 0	FP(domain) 0	TP(url) 7	TP(domain) 7	^\w{5,6}\.www\.7snyj\.download$
batch index 4	sample index 25	FP(url) 0	FP(domain) 0	TP(url) 97	TP(domain) 97	^\w{6,10}\.iw3fhwsvw\.com$
batch index 6	sample index 20	FP(url) 0	FP(domain) 0	TP(url) 18	TP(domain) 18	^\w{5,6}\.kegwc\.ox28e\.download$
batch index 3	sample index 28	FP(url) 0	FP(domain) 0	TP(url) 11	TP(domain) 11	^\w{5,6}\.yowu2\.vtygs\.win$
batch index 5	sample index 23	FP(url) 0	FP(domain) 0	TP(url) 14	TP(domain) 14	^\w{6}\.0e82c\.wg5li\.download$
batch index 7	sample index 18	FP(url) 0	FP(domain) 0	TP(url) 4	TP(domain) 4	^\w{5,6}\.o147im\.dllm3\.www\.q3537\.download$
batch index 1	sample index 32	FP(url) 0	FP(domain) 0	TP(url) 9	TP(domain) 9	^\w{5,6}\.t6i

...

batch index 7	sample index 519	FP(url) 0	FP(domain) 0	TP(url) 20	TP(domain) 17	^\w{5,6}\.8ok2u\.hfusi\.win$
batch index 2	sample index 539	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^thebetterandreliable2updating\.website$
batch index 1	sample index 545	FP(url) 0	FP(domain) 0	TP(url) 11	TP(domain) 11	^\w{5,6}\.xbbbp\.wvk6m\.download$
batch index 5	sample index 537	FP(url) 0	FP(domain) 0	TP(url) 11	TP(domain) 11	^\w{5,6}\.4qwwy\.20s06\.download$
batch index 0	sample index 547	FP(url) 0	FP(domain) 0	TP(url) 52	TP(domain) 52	^\w{3}\.rygdd\.cn$
batch index 4	sample index 540	FP(url) 0	FP(domain) 0	TP(url) 10	TP(domain) 10	^\w{6}\.weww0\.vjjjk\.faith$
batch index 6	sample index 538	FP(url) 0	FP(domain) 0	TP(url) 15	TP(domain) 15	^\w{5,6}\.g42yk\.20s06\.download$
batch index 3	sample index 544	FP(url) 0	FP(domain) 0	TP(url) 0	TP(domain) 0	^www\.rtyszz\.com$
batch index 7	sample index 520	FP(url) 0	FP(domain) 0	TP(url) 12	TP(domain) 9	^\w{5,6}\.2zah4\.www\.vjjg\.faith$
batch index 2	sample

batch index 5	sample index 546	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^theonlygoodplacecontent\.(:?win|bid)$
batch index 1	sample index 554	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 1	^paypal\.com\.my\.cgi\.bin\.webscr\.cmd\.login\.submit\.dispatch\.5885d80a13c0db1f8e263663d3faee8\.signssignssigns\.com$
batch index 2	sample index 548	FP(url) 0	FP(domain) 0	TP(url) 13	TP(domain) 13	^(:?developedsoftforfreeandset|developed-softforfreenow|developedsoftforfreenew|developedsoftforfreenewer)\.(:?pw|club|top|bid)$
batch index 7	sample index 528	FP(url) 0	FP(domain) 0	TP(url) 2849	TP(domain) 2554	^\w{4,6}\.15525\.www\.gtiuo\.win$
batch index 4	sample index 549	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^safeandpreparedupgradesystems\.download$
batch index 3	sample index 553	FP(url) 0	FP(domain) 0	TP(url) 29	TP(domain) 24	^\w{5,6}\.hco6y3\.fhyh4s\.hsx12\.www\.vtygs\.win$
batch index 0	sample index 556	FP(url) 0	FP(domain) 0	TP(url) 14	TP(domain) 13	^\w{5,6}\.6a1ju\.www\.dur34\.win$


batch index 5	sample index 555	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^readyandpreparedsetuptoupdates\.(:?pw|pro)$
batch index 3	sample index 561	FP(url) 0	FP(domain) 0	TP(url) 8	TP(domain) 8	^(:?goodplace-contentsforcentralsitealways|goodplace-contentforcentralsiteclear)\.(:?win|bid|trade|download|date|review)$
batch index 0	sample index 564	FP(url) 0	FP(domain) 0	TP(url) 25	TP(domain) 24	^\w{5,6}\.uquu8\.josid\.faith$
batch index 6	sample index 555	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^24checkforfreesystemslikepcandmacs\.(:?club|pro)$
batch index 2	sample index 557	FP(url) 0	FP(domain) 0	TP(url) 7	TP(domain) 7	^\w{6}\.wuy6z\.www\.vgkj\.faith$
batch index 1	sample index 563	FP(url) 0	FP(domain) 0	TP(url) 67	TP(domain) 58	^\w{5}\.eaajc\.cn$
batch index 7	sample index 537	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^thebigandgreat2upgrades\.win$
batch index 5	sample index 556	FP(url) 0	FP(domain) 0	TP(url) 3	TP(domain) 3	^(:?theserviceforallupdating|theserviceforallupd

batch index 5	sample index 564	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^installupgradesetforpcandmac\.download$
batch index 3	sample index 570	FP(url) 0	FP(domain) 0	TP(url) 19	TP(domain) 19	^\w{4,6}\.ci0a6\.z703h\.download$
batch index 4	sample index 566	FP(url) 0	FP(domain) 0	TP(url) 11	TP(domain) 11	^\w{5,6}\.7hf02\.www\.wipss\.win$
batch index 7	sample index 546	FP(url) 0	FP(domain) 0	TP(url) 12	TP(domain) 12	^\w{5,6}\.979nx\.uewi\.win$
batch index 6	sample index 564	FP(url) 0	FP(domain) 0	TP(url) 7	TP(domain) 7	^\w{6}\.z91j7\.fbhwi6\.win$
batch index 2	sample index 566	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^edu\.nrgof\.cn$
batch index 1	sample index 572	FP(url) 0	FP(domain) 0	TP(url) 56	TP(domain) 38	^\w{5,6}\.fklfd\.faith$
batch index 5	sample index 565	FP(url) 0	FP(domain) 0	TP(url) 18	TP(domain) 18	^\w{5,6}\.i8kq6\.ar9r6\.download$
batch index 4	sample index 567	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^15\.0dzucw\.qmap3\.www\.hugew0\.win$
batch index 3	sampl

2018-04-21 12:40:53 file_io.py [line:255] DEBUG	Check Result has been dump	../../data/EXP_domain/regex_result.txt


In [48]:
df = regex_domain.domain_regex_publish(
                                  "../../data/EXP_domain/regex_result.txt",
                                  "../../data/EXP_domain/regex_publish.txt",
                                  publish_score_thresh = 0.4,
                                  publish_ratio = 10,
                                  publish_fp_thresh = 5000,
                                  publish_tp_thresh = 0)

2018-04-24 12:49:18 file_io.py [line:276] DEBUG	check Data has been loaded	../../data/EXP_domain/regex_result.txt
2018-04-24 12:49:39 regex_domain.py [line:460] DEBUG	regular expression publish	4547
2018-04-24 12:49:39 file_io.py [line:207] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_domain/regex_publish.txt
2018-04-24 12:49:39 file_io.py [line:212] DEBUG	Regex has been dump	../../data/EXP_domain/regex_publish.txt


In [8]:
predict_malicious, predict_dict, predict_dict_detail = regex_domain.malicious_domain_predict(
                      input_file_path = url_2,
                      regex_file_path = "../../data/EXP_domain2/regex_publish.txt")

2018-04-25 11:28:00 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_domain2/regex_publish.txt


batch index 0	sample index 0	hit domain 0	^\w{4,6}\.tti41k\.64ryqf\.9s40r\.www\.vird\.win$
batch index 1	sample index 0	hit domain 0	^[A-Za-z]{3,7}up[A-Za-z]{3,5}\.(:?streamingupdatesyouneedtowatchallthismonth|streamingupdatesyou-needtowatchallthismonth)\.(:?review|stream)$
batch index 0	sample index 1	hit domain 0	^\w{6}\.p9fev\.www\.fgi5\.win$
batch index 1	sample index 1	hit domain 0	^\w{4,6}\.izlinix\.com$
batch index 1	sample index 2	hit domain 0	^\w{4,6}\.\w{4,6}\.\w{3,6}\.\w{5,6}\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 0	sample index 2	hit domain 0	^\w{6}\.wuo6c\.f8d9\.faith$
batch index 0	sample index 3	hit domain 0	^\w{4,6}\.\w{5,6}\.47160\.ftt\.(:?y3mx35|11xpwb)\.wymigx\.zjwpq1\.kskddh\.co0qvm\.uul0jd\.dllm3\.www\.q3537\.download$
batch index 1	sample index 3	hit domain 0	^(:?always4evernewesoftwares|always4evernewestupdates)\.(:?win|top)$
batch index 2	sample index 0	hit domain 0	^\w{6}\.d5nh7\.mi842\.download$
batch index 0	sample index 4	hit dom

batch index 4	sample index 13	hit domain 0	^\w{5,6}\.pf47m\.www\.josid\.faith$
batch index 0	sample index 19	hit domain 1	^\w{4,15}\.000webhostapp\.com$
batch index 3	sample index 13	hit domain 0	^\w{5}\.42783\.onysk\.www\.ngtuh\.win$
batch index 2	sample index 19	hit domain 0	^\w{5,6}\.4206e\.fmirir\.win$
batch index 1	sample index 21	hit domain 0	^\w{6}\.y4w6w\.vhty\.win$
batch index 5	sample index 7	hit domain 0	^(:?8bxh18|u3wcp2)\.(:?azrk10|8gihy6)\.1per0v\.598kgl\.1p62u\.www\.f4k5\.faith$
batch index 4	sample index 14	hit domain 0	^[A-Za-z]{6,7}\.zgjjmsw\.com$
batch index 6	sample index 3	hit domain 0	^(:?yourbettersofisticatedtoupgrades|yourbettersofisticatedtoupgrading|yourbettersofisticatedtoupgrade)\.(:?website|win|bid|top)$
batch index 7	sample index 0	hit domain 0	^\w{4,6}\.eyqcq\.5niyv\.download$
batch index 6	sample index 4	hit domain 0	^\w{6}\.g668w\.5niyv\.download$
batch index 2	sample index 20	hit domain 0	^\w{5,6}\.www\.wioi\.faith$
batch index 1	sample index 22	hit d

batch index 1	sample index 34	hit domain 0	^verify\.facebook\.com-------mobile---read---new--terms--\d{9}\.peraltek\.com$
batch index 5	sample index 19	hit domain 0	^(:?xrc42m|mkr1gz)\.m\.hnkangermei\.com$
batch index 3	sample index 25	hit domain 0	^\w{4,6}\.91408\.30287\.yhawac\.uul0jd\.dllm3\.www\.q3537\.download$
batch index 2	sample index 32	hit domain 0	^app\.(:?viideossexy|videosdks)\.com$
batch index 6	sample index 14	hit domain 0	^\w{5,6}\.oc5vea\.15525\.www\.gtiuo\.win$
batch index 4	sample index 29	hit domain 0	^eee\.\w{5,6}\.15525\.www\.gtiuo\.win$
batch index 7	sample index 11	hit domain 0	^(:?52134|70077|sf|ftt)\.(:?67f4lg|hv67h1|okild|oiv47b)\.1188\.3ewzy\.cnkdjx\.win$
batch index 3	sample index 26	hit domain 0	^fd[A-Za-z]{1}\.uglyas\.com$
batch index 2	sample index 33	hit domain 0	^\w{4,6}\.www\.ox28e\.download$
batch index 1	sample index 35	hit domain 0	^\w{5,6}\.qcs0s\.3s0lp\.download$
batch index 0	sample index 32	hit domain 0	^www\.(:?secure2|secure1)\.appelid-appel\

batch index 4	sample index 39	hit domain 0	^\w{6}\.xnx1t\.geie\.faith$
batch index 0	sample index 43	hit domain 0	^\w{5,6}\.9bclw\.www\.lwzbf\.download$
batch index 3	sample index 38	hit domain 0	^\w{5,6}\.3tp7h\.qhpq6\.download$
batch index 4	sample index 40	hit domain 0	^\w{2,3}zz[A-Za-z]{0,1}\.k5scm\.migox\.xyz$
batch index 5	sample index 33	hit domain 0	^\w{5,6}\.g8mo4\.nfu02\.win$
batch index 7	sample index 21	hit domain 0	^\w{5,6}\.zgry7\.euiws\.win$
batch index 6	sample index 25	hit domain 0	^\w{5,6}\.ruhfu\.faith$
batch index 2	sample index 46	hit domain 0	^\w{5,6}\.lfchanraomo\.com$
batch index 0	sample index 44	hit domain 0	^\w{5,6}\.kiwme\.hfusi\.win$
batch index 1	sample index 48	hit domain 0	^\w{5,6}\.ywavh\.download$
batch index 3	sample index 39	hit domain 0	^(:?63432|8182)\.2399\.vbs02\.8guwuz\.3942\.i2j0z\.www\.dioe\.win$
batch index 4	sample index 41	hit domain 0	^\w{5,6}\.co48g\.wioi\.faith$
batch index 7	sample index 22	hit domain 0	^\w{5,6}\.ias8a\.li740\.download$

...

batch index 2	sample index 525	hit domain 0	^\w{5,6}\.1l7dr\.p30qd\.download$
batch index 6	sample index 510	hit domain 0	^\w{6}\.(:?80131|28o3fv)\.2vmhx5\.2qkku\.jsid\.win$
batch index 5	sample index 497	hit domain 0	^(:?vmf7vs|j3ws2l|32707|e9v1kd)\.m\.gzlaishirong\.com$
batch index 1	sample index 533	hit domain 0	^174\.(:?138|136)\.57\.170$
batch index 3	sample index 505	hit domain 0	^\w{6}\.nxckk6\.15525\.www\.gtiuo\.win$
batch index 7	sample index 486	hit domain 0	^\w{5,6}\.wsvjk\.faith$
batch index 4	sample index 503	hit domain 0	^\w{5,6}\.www\.323444\.win$
batch index 2	sample index 526	hit domain 0	^\w{4,6}\.uikyi\.frui\.win$
batch index 0	sample index 524	hit domain 0	^\w{5,6}\.8i0c2\.6wt0v\.download$
batch index 6	sample index 511	hit domain 0	^\w{5,6}\.tdn5x\.f7i3\.win$
batch index 1	sample index 534	hit domain 0	^[A-Za-z]{5,11}\.ekmdigitalvault\.com$
batch index 3	sample index 506	hit domain 0	^\w{6}\.qv4zh\.www\.fkjjd\.win$
batch index 2	sample index 527	hit domain 0	^000\w

batch index 7	sample index 497	hit domain 0	^\w{5,6}\.426ks\.ruyt\.win$
batch index 0	sample index 537	hit domain 0	^maximalforupgrade\.trade$
batch index 2	sample index 539	hit domain 0	^(:?developedsoftforfreeandset|developed-softforfreenow|developedsoftforfreenew|developedsoftforfreenewer)\.(:?pw|club|top|bid)$
batch index 4	sample index 515	hit domain 0	^\w{2,5}\.cqxh2013\.com$
batch index 6	sample index 523	hit domain 0	^\w{5,6}\.51ayk\.www\.cnkdjx\.win$
batch index 3	sample index 518	hit domain 0	^[^\.]{7,12}\.blogspot\.com\.es$
batch index 1	sample index 547	hit domain 0	^\w{5,6}\.wyqi0\.vjhf\.win$
batch index 0	sample index 538	hit domain 0	^\w{6}\.(:?42027|rencai)\.sortlh\.15525\.www\.gtiuo\.win$
batch index 2	sample index 540	hit domain 0	^(:?www|24pccheck)\.the-readysystems4contents\.download$
batch index 7	sample index 498	hit domain 0	^support\.na\.square\.login-[A-Za-z]{4}\.usa\.cc$
batch index 5	sample index 507	hit domain 0	^[A-Za-z]{7,11}\.blogspot\.com\.es$
batch inde

batch index 6	sample index 535	hit domain 0	^\w{5,6}\.ao4wu\.fnhu04\.win$
batch index 7	sample index 508	hit domain 0	^\w{5,6}\.z4qux\.download$
batch index 5	sample index 518	hit domain 0	^\w{5,6}\.777rp\.fifjsi\.win$
batch index 2	sample index 552	hit domain 0	^(:?developedsoftforfreeandset|developedsoftforfreeandgo|developedsoftforfreenewer)\.(:?info|pw|club|top)$
batch index 0	sample index 550	hit domain 0	^\w{4,6}\.cqmyy\.vjjjk\.faith$
batch index 6	sample index 536	hit domain 0	^\w{5,6}\.www\.dhuif\.win$
batch index 3	sample index 529	hit domain 0	^\w{4,6}\.n1dt9\.ea8nw\.download$
batch index 4	sample index 526	hit domain 0	^\w{5,6}\.nv1zf\.f84s\.win$
batch index 5	sample index 519	hit domain 0	^(:?44k1rk|f4l5dn|3eztda)\.(:?69284|75231|79530)\.(:?aqejc9|pqv5zh|svuj1l)\.uul0jd\.dllm3\.www\.q3537\.download$
batch index 7	sample index 509	hit domain 0	^\w{5,6}\.fhj106\.15525\.www\.gtiuo\.win$
batch index 4	sample index 527	hit domain 0	^\w{4,6}\.t997v\.wuus\.win$
batch index 2	sampl

batch index 7	sample index 521	hit domain 0	^\w{5,6}\.2l6yl\.www\.ruyt\.win$
batch index 5	sample index 532	hit domain 0	^\w{5,6}\.gm4we\.smy4o\.download$
batch index 6	sample index 550	hit domain 0	^\w{5,6}\.og2ay\.dichanpinglun\.win$
batch index 4	sample index 539	hit domain 0	^safeandpreparedupgradesystems\.download$
batch index 3	sample index 541	hit domain 0	^\w{4,6}\.u420y\.8ae6q\.download$
batch index 2	sample index 567	hit domain 0	^\w{5,6}\.ffrxt\.gigwe\.download$
batch index 0	sample index 564	hit domain 0	^\w{5,6}\.(:?boke|o6nj2d)\.2m59hd\.xx37d\.223222\.win$
batch index 7	sample index 522	hit domain 0	^\w{5,6}\.r319d\.dhuif\.win$
batch index 6	sample index 551	hit domain 0	^\w{5,6}\.rvk04\.www\.smy4o\.download$
batch index 4	sample index 540	hit domain 0	^\w{1,5}\.zchtmdy\.com$
batch index 3	sample index 542	hit domain 0	^\w{3}\.nrlgn\.cn$
batch index 6	sample index 552	hit domain 0	^\w{5,6}\.y6ai0\.vbyd\.win$
batch index 5	sample index 533	hit domain 0	^(:?h0elrn|7spe1s|m1

batch index 4	sample index 558	hit domain 0	^\w{5,6}\.yoam8\.vjdod\.win$
batch index 7	sample index 539	hit domain 0	^\w{6,7}\.zz550rf0nz\.com$
batch index 3	sample index 560	hit domain 0	^\w{4,6}\.ci0a6\.z703h\.download$
batch index 5	sample index 550	hit domain 0	^(:?down1|down3)\.(:?pmdown|3ddown)\.com$
batch index 4	sample index 559	hit domain 0	^\w{4,6}\.6826k\.giri\.win$
batch index 7	sample index 540	hit domain 0	^\w{6}\.bhj75\.mi842\.download$
batch index 3	sample index 561	hit domain 0	^\w{4,6}\.\w{5,6}\.\w{5,6}\.\w{5,6}\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 4	sample index 560	hit domain 0	^\w{5,6}\.cmne2v\.48eso\.dichanpinglun\.win$
batch index 5	sample index 551	hit domain 0	^\w{2,4}\.huihomes\.com$
batch index 7	sample index 541	hit domain 0	^nysvp\.\w{5,6}\.15525\.www\.gtiuo\.win$
batch index 3	sample index 562	hit domain 0	^\w{4,6}\.87390\.22305\.ytbyzp\.dllm3\.www\.q3537\.download$
batch index 4	sample index 561	hit domain 0	^\w{5,6}\.\w{4,6

In [9]:
len(predict_malicious)

1539

In [10]:
for k, v in predict_dict_detail.iteritems():
    print "%40s\t\tdomain:%d\turl:%d" %(k, len(v), sum([len(_) for _ in v]))

             ^down\w{0,6}\.52zsoft\.com$		domain:1	url:1
               ^bd\d{1,2}\.52lishi\.com$		domain:1	url:2
            ^\d{5}\.url\.9xiazaiqi\.com$		domain:29	url:50
                 ^\w{3,4}\.charrem\.com$		domain:3	url:8
             ^\d{4,5}\.url\.tudown\.com$		domain:1	url:1
               ^\d{4,5}\.url\.7wkw\.com$		domain:7	url:8
    ^[A-Za-z]{2,3}\.mingpinhuishou\.com$		domain:2	url:2
             ^[A-Za-z]{2}\.yunmiss\.net$		domain:1	url:1
          ^\w{4,15}\.000webhostapp\.com$		domain:1	url:1
             ^\d{4,8}\.url\.246546\.com$		domain:116	url:289
    ^[A-Za-z]{5,13}\.000webhostapp\.com$		domain:1	url:1
               ^\w{4,6}\.downyouxi\.com$		domain:2	url:2
            ^\w{3,4}\.kaijiaweishi\.com$		domain:2	url:24
              ^\d{4,5}\.url\.222bz\.com$		domain:36	url:47
                   ^\w{2,3}\.mqego\.com$		domain:10	url:46
          ^(:?xz|c|w|6)\.gyingyuan\.com$		domain:1	url:1
             ^mitaosp\d{3}\.17arch\.com$		domain:5	url:8
               ^\d{5

In [4]:
def collect_tp(regex_list, predict_dict_detail):
    mal_url = []
    for regex in regex_list:
        mal_url.extend([j for i in predict_dict_detail[regex] for j in i])
    return mal_url

In [11]:
mal_url = collect_tp(["^\d{6}\.sdyintong\.com$"], predict_dict_detail)

In [14]:
safe_urls = load_urls("../../data/sangfor/safe_0419_mod.csv")

2018-04-25 11:29:30 file_io.py [line:52] DEBUG	URLs Count:	1455678


In [26]:
safe_urls_modify5 = []
cc = 0
for i in safe_urls_modify4:
    if "url.222bz.com" in i:
        cc+=1
        continue
    safe_urls_modify5.append(i)
print cc

211


In [27]:
dump_urls(safe_urls_modify5, "../../data/sangfor/safe_0419_mod.csv")

2018-04-25 12:54:59 file_io.py [line:24] DEBUG	OLD DATA FIND! REMOVING	../../data/sangfor/safe_0419_mod.csv
2018-04-25 12:55:02 file_io.py [line:33] DEBUG	URLs has been dump	../../data/sangfor/safe_0419_mod.csv


In [32]:
df = url_forest.analyze_domain_batch(safe_url_domain, safe_forest, mal_forest, verbose = True)

step:0	uncertain:0	predict_mal:0
123.249.9.78
step:5000	uncertain:676	predict_mal:10
222.186.172.44
step:10000	uncertain:1288	predict_mal:19
221.5.109.20
step:15000	uncertain:1949	predict_mal:31
148.251.131.12
step:20000	uncertain:2601	predict_mal:39
180.97.220.14
111.230.39.167
step:25000	uncertain:3294	predict_mal:48
222.186.51.186
47.91.244.105
step:30000	uncertain:3963	predict_mal:56
185.10.68.202
step:35000	uncertain:4673	predict_mal:69
203.207.195.153
step:40000	uncertain:5387	predict_mal:77
210.73.154.100
222.186.172.41
210.42.122.221
step:45000	uncertain:6074	predict_mal:84
211.69.16.29
181.214.87.241
step:50000	uncertain:6813	predict_mal:90
120.78.203.230
step:55000	uncertain:7478	predict_mal:94
84.73.251.157
202.202.155.137
52.175.207.110
step:60000	uncertain:8123	predict_mal:102
step:65000	uncertain:8790	predict_mal:115
222.186.50.227
step:70000	uncertain:9487	predict_mal:123
222.186.191
step:75000	uncertain:10160	predict_mal:129
118.184.50.24
step:80000	uncertain:10815	pred

In [47]:
df_mal = df.loc[df.domain_reputation>10]
df_mal

,domain_reputation,url
704,14.405035,down05067522.xdown4.com
3040,230.116473,polycache.000webhostapp.com
4202,14.405035,down05072973.xdown4.com
9747,31.536926,inovatherm.free.fr
10296,230.116473,hghhghghgh.000webhostapp.com
11679,11.735331,dx155.downyouxi.com
14431,14.405035,down05067398.xdown4.com
20390,16.841579,19473.url.9xiazaiqi.com
23559,18.407960,dianxin8.52zsoft.com
25086,14.405035,down05067460.xdown4.com


In [3]:
url_forest.analyze_domain(safe_forest, mal_forest, domain="adsf.ebutscdo.cn", verbose = True)

NameError: name 'safe_forest' is not defined

In [55]:
import validate
reload(validate)
from validate import ValidateWithVT

In [56]:
valid_obj = ValidateWithVT()
df = valid_obj.download_file(predict_malicious, 
                             "/home/sparrow/sparrow/data/edr_test/domain")

2018-04-24 12:52:15 validate.py [line:49] DEBUG	mkdir /home/sparrow/sparrow/data/edr_test/domain
2018-04-24 12:52:17 validate.py [line:22] DEBUG	index	4	http://20102.xc.cangpie.com/xiaz/%E5%A4%A7%E7%A6%B9%E6%B2%BB%E6%B0%B4%E7%B2%BE%E5%BD%A9PPT%E8%AF%BE%E4%BB%B6%E4%B8%8B%E8%BD%BD@257_97134.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:52:17 validate.py [line:22] DEBUG	index	7	http://20019.xc.cangpie.com/xiaz/%A1%B6%C8%CB%B9%A4%C9%D9%C5%AE3%A1%B7%C3%E2%B0%B2%D7%B0%D3%B2%C5%CC%B0%E6@84_208782.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:52:17 validate.py [line:22] DEBUG	index	5	http://19963.xc.cangpie.com/xiaz/HPLaserJetP1108@311_11980.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:52:17 validate.py [line:22] DEBUG	index	1	http://19935.xc.cangpie.com/xiaz/%E7%94%B5%E5%BD%B1%E5%8E%89%E5%AE%B3%E4%BA%86%E6%88%91%E7%9A%84%E5%9B%BD%E9%AB%98%E6%B8%85%E5%AE%8C%E6%95%B4%E7%89%88%E7%99%BE

2018-04-24 12:52:19 validate.py [line:22] DEBUG	index	25	http://19934.xc.cangpie.com/xiaz/%E6%83%A0%E6%99%AEHPOfficejet100@383_395.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:52:19 validate.py [line:22] DEBUG	index	29	http://20103.url.246546.com/xiaz/%E9%87%8D%E7%A3%85%E6%9D%A5%E8%A2%AD91%E7%A7%A6%E5%85%88%E7%94%9F%E7%AC%AC%E5%9B%9B%E9%83%A8%E7%94%B5%E5%BD%B1%E9%99%A2%E5%A5%B3%E7%A5%9E%E7%90%AA%E7%90%AA%E5%8D%AB%E7%94%9F%E9%97%B4%E6%BF%80%E6%83%85%E5%95%AA%E5%95%AA26%E5%88%86%E9%92%9F%E9%AB%98%E6%B8%85%E6%97%A0%E6%B0%B4%E5%AE%8C%E6%95%B4%E7%89%88.mkv_%E7%99%BE%E5%BA%A6%E7%BD%91%E7%9B%98-%E7%BD%91%E7%9B%98007@554_2.exe	d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c
2018-04-24 12:52:19 validate.py [line:22] DEBUG	index	40	http://20101.xc.cangpie.com/xiaz/Fabmaster%E4%B8%AD%E6%96%87%E7%A0%B4%E8%A7%A3%E7%89%88(%E9%99%84%E6%95%99%E7%A8%8B)@251_46107.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:52:

2018-04-24 12:52:24 validate.py [line:22] DEBUG	index	67	http://19890.xc.cangpie.com/xiaz/%E6%A8%A1%E6%8B%9F%E4%BA%BA%E7%94%9F4%E5%AE%89%E5%8D%93%E7%A0%B4%E8%A7%A3%E7%89%88%20%E7%94%B5%E8%84%91%E7%89%88@58_28741.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:52:24 validate.py [line:22] DEBUG	index	70	http://19960.xc.cangpie.com/xiaz/ccproject%E7%BD%91%E7%BB%9C%E5%9B%BE%E7%BB%98%E5%88%B6%E8%BD%AF%E4%BB%B6%E7%A0%B4%E8%A7%A3%E7%89%88@141_197132.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:52:24 validate.py [line:22] DEBUG	index	71	http://20126.xc.cangpie.com/xiaz/%E8%89%AF%E6%B3%95app%E7%94%B5%E8%84%91%E7%89%882018%E6%9C%80%E6%96%B0%E7%89%88@212_333245.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:52:24 validate.py [line:22] DEBUG	index	68	http://20102.xc.cangpie.com/xiaz/Office2010@452_24189.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:52:24

2018-04-24 12:52:29 validate.py [line:22] DEBUG	index	49	http://19715.xc.cangpie.com/down/SeewoLink%E5%B8%8C%E6%B2%83%E6%8E%88%E8%AF%BE%E5%8A%A9%E6%89%8B%E7%94%B5%E8%84%91%E7%89%883.0.12.2956%E5%AE%98%E6%96%B9%E5%85%8D%E8%B4%B9%E7%89%88@158_185355.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:52:29 validate.py [line:22] DEBUG	index	96	http://566092.sdyintong.com/qq001002/300100_927545.apk	dfef3ef9eee1e040d68620555bfb356bd6a43428cfb04a12842615cd828ed2b1
2018-04-24 12:52:29 validate.py [line:22] DEBUG	index	98	http://19697.xc.cangpie.com/xiaz/%E5%B0%BE%E8%A1%8C44%EF%BC%9A%E8%99%9A%E6%8B%9F%E8%BF%BD%E8%B8%AA%E8%80%85@84_213731.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:52:29 validate.py [line:22] DEBUG	index	99	http://20044.xc.cangpie.com/xiaz/EpsonLQ-735K@311_12182.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:52:29 validate.py [line:22] DEBUG	index	95	http://20123.xc.cangpie

2018-04-24 12:52:30 validate.py [line:22] DEBUG	index	107	http://19960.xc.cangpie.com/xiaz/%E5%8F%8D%E6%81%90%E7%B2%BE%E8%8B%B11.6@382_4476.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:52:30 validate.py [line:22] DEBUG	index	103	http://19966.xc.cangpie.com/xiaz/%E9%AD%94%E5%85%BD%E4%BA%89%E9%9C%B83%E5%86%B0%E5%B0%81%E7%8E%8B%E5%BA%A71.24e%E4%B8%8B%E8%BD%BD-%E5%8D%95%E6%9C%BA%E6%B8%B8%E6%88%8F-52pk%E6%B8%B8%E6%88%8F%E4%B8%8B%E8%BD%BD%E4%B8%AD%E5%BF%83@168_58.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:52:30 validate.py [line:22] DEBUG	index	111	http://20034.xc.cangpie.com/xiaz/%E4%B9%9D%E9%A6%96%E6%AD%8C/%E6%83%85%E6%AC%B2%E4%B9%9D%E6%AD%8C%E9%AB%98%E6%B8%85_%E7%99%BE%E5%BA%A6%E4%BA%91%E7%BD%91%E7%9B%98_%E5%9C%A8%E7%BA%BF%E8%A7%82%E7%9C%8B_%E8%BF%85%E9%9B%B7%E4%B8%8B%E8%BD%BD@439_2.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:52:30 validate.py [line:22] DEBUG	index	112	http

...

2018-04-24 12:55:27 validate.py [line:22] DEBUG	index	1400	http://19939.xc.cangpie.com/xiaz/%E4%BE%A0%E7%9B%97%E7%8C%8E%E8%BD%A6%E6%89%8B%EF%BC%9A%E7%BD%AA%E6%81%B6%E9%83%BD%E5%B8%82%E4%B8%AD%E6%96%87%E7%A1%AC%E7%9B%98%E7%89%88@160_476.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:55:27 validate.py [line:22] DEBUG	index	1410	http://20029.url.246546.com/xiaz/CDRX4%E7%B2%BE%E7%AE%80%E7%89%88%E5%AE%89%E8%A3%85%E5%8C%85.zip_%E7%99%BE%E5%BA%A6%E7%BD%91%E7%9B%98-%E7%BD%91%E7%9B%98007@554_2.exe	d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c
2018-04-24 12:55:28 validate.py [line:22] DEBUG	index	1397	http://20052.xc.cangpie.com	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:55:28 validate.py [line:22] DEBUG	index	1404	http://20054.xc.cangpie.com/xiaz/MicrosoftPowerPointViewer2010%E7%AE%80%E4%BD%93%E4%B8%AD%E6%96%87%E7%89%88@696_180532.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-0

2018-04-24 12:55:32 validate.py [line:22] DEBUG	index	1442	http://11055.xc.cangpie.com/xiaz/%E3%80%8A%E4%BE%A0%E7%9B%97%E9%A3%9E%E8%BD%A6%E7%BD%AA%E6%81%B6%E9%83%BD%E5%B8%82%E3%80%8B%E4%B8%AD%E6%96%87%E5%AE%8C%E5%85%A8%E7%89%88%E6%B8%B8%E6%88%8F%E4%B8%8B%E8%BD%BD@322_706.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:55:32 validate.py [line:22] DEBUG	index	1439	http://19928.xc.cangpie.com/down/ps6%E5%BA%8F%E5%88%97%E5%8F%B7%E6%B3%A8%E5%86%8C%E6%9C%BA@133_45017.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:55:32 validate.py [line:22] DEBUG	index	1433	http://19908.xc.cangpie.com/xiaz/Office2010@418_24189.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:55:32 validate.py [line:22] DEBUG	index	1444	http://19930.xc.cangpie.com/xiaz/%E8%81%94%E6%83%B3LenovoLJ2400L@311_11837.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:55:32 validate.py [line:22] DEB

2018-04-24 12:55:38 validate.py [line:22] DEBUG	index	1475	http://19935.xc.cangpie.com	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:55:38 validate.py [line:22] DEBUG	index	1472	http://19928.xc.cangpie.com/xiaz/%E5%BE%AE%E4%BF%A1%E5%9B%BE%E6%A0%87%E4%B8%8B%E8%BD%BD@177_86152.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:55:38 validate.py [line:22] DEBUG	index	1477	http://19919.url.246546.com/xiaz/directx_nov2008_redist@241_149076.exe	d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c
2018-04-24 12:55:39 validate.py [line:22] DEBUG	index	1451	http://298414.sdyintong.com/qq001002/900200_624166.apk	dfef3ef9eee1e040d68620555bfb356bd6a43428cfb04a12842615cd828ed2b1
2018-04-24 12:55:39 validate.py [line:22] DEBUG	index	1450	http://248478.sdyintong.com/qq001002/70200_038088.apk	dfef3ef9eee1e040d68620555bfb356bd6a43428cfb04a12842615cd828ed2b1
2018-04-24 12:55:39 validate.py [line:22] DEBUG	index	1479	http://1995

2018-04-24 12:55:44 validate.py [line:22] DEBUG	index	1502	http://19934.url.246546.com/xiaz/%E8%81%94%E6%83%B3%E9%A9%B1%E5%8A%A8%E4%B8%80%E9%94%AE%E5%AE%89%E8%A3%85(%E8%81%94%E6%83%B3%E4%B9%90%E9%A9%B1%E5%8A%A8)v3.50.1225%E5%AE%98%E6%96%B9%E5%AE%89%E8%A3%85%E7%89%88@317_16859.exe	d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c
2018-04-24 12:55:44 validate.py [line:22] DEBUG	index	1509	http://ddup.kaijiaweishi.com/public/package/201802/Other/KaiJiaWeiShiSetup0411@uqd.exe	8c267fb8c2051d010284afecf78a544145dff5c780aa31f84337ebf944913bba
2018-04-24 12:55:44 validate.py [line:22] DEBUG	index	1511	http://19929.xc.cangpie.com/down/%E9%87%91%E5%B1%B1%E6%B8%B8%E4%BE%A0v%E4%BF%AE%E6%94%B9%E5%99%A8@133_21885.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:55:44 validate.py [line:22] DEBUG	index	1492	http://19689.url.9xiazaiqi.com/xiaz/%E5%8C%96%E5%AD%A6%E9%87%91%E6%8E%92(%E6%96%B9%E7%A8%8B%E5%BC%8F%E9%85%8D%E5%B9%B3%E8%BD%AF%E4%BB%B6)v12.0%E4%B8

2018-04-24 12:55:48 validate.py [line:22] DEBUG	index	1536	http://19934.xc.cangpie.com/xiaz/%E5%B9%BF%E5%8F%91%E8%AF%81%E5%88%B8%E6%B8%AF%E8%82%A1%E9%80%9A%E4%B8%93%E4%B8%9A%E7%89%88@271_136821.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:55:48 validate.py [line:22] DEBUG	index	1538	http://20099.xc.cangpie.com/xiaz/yuweij%E5%AD%97%E4%BD%93@68_254852.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:55:49 validate.py [line:22] DEBUG	index	1534	http://xza.mqego.com/ccleaner_plus.zip	ec9b1e89291202a413863806cf69b64bb7b68d6e5ad074a623b217ba8be53d5b
2018-04-24 12:55:58 validate.py [line:22] DEBUG	index	1446	http://mitaosp823.17arch.com/%E5%A5%B3l%E5%AA%9B1%E7%9B%B4l%E6%92%AD_0200-app	ada861a325af53d85f76a2f5e70d215df3e2c4dbf4a3bb7740f24d0c21a2a4e8


In [59]:
df.to_csv("../../data/EXP_domain/df_res_url.csv", index = False)

In [61]:
sha256_list = valid_obj.upload_file_to_vt("/home/sparrow/sparrow/data/edr_test/domain")

2018-04-24 12:57:00 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-24 12:57:07 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-24 12:57:08 validate.py [line:141] DEBUG	success to upload file to VT 3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-24 12:57:08 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-24 12:57:11 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-24 12:57:11 validate.py [line:141] DEBUG	success to upload file to VT d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c
2018-04-24 12:57:11 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): ww

2018-04-24 12:58:08 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-24 12:58:08 validate.py [line:141] DEBUG	success to upload file to VT 8e47a0190b5f9e4cc8498a6af5fb5cc61337d31e4d5aec9c27e68cbc749bf09c
2018-04-24 12:58:08 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-24 12:58:10 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-24 12:58:10 validate.py [line:141] DEBUG	success to upload file to VT aa8e04cd4cf56392c2301154565f273802130d1362c833dc167f7cec815cb679
2018-04-24 12:58:10 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-24 12:58:20 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST 

2018-04-24 13:02:24 validate.py [line:141] DEBUG	success to upload file to VT d3b1894fe02d4f4f5ec227f6f5ee0163d427899ea16141589b09454043ba463e
2018-04-24 13:02:25 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-24 13:02:31 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-24 13:02:31 validate.py [line:141] DEBUG	success to upload file to VT 5d585333178f18b68049eec3294adb1deed85275f3b9ea31d96ac57d3ad8ecfe
2018-04-24 13:02:31 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-24 13:02:35 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-24 13:02:35 validate.py [line:141] DEBUG	success to upload file to VT 8b4331b23597cf

2018-04-24 13:05:12 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-24 13:05:14 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-24 13:05:14 validate.py [line:141] DEBUG	success to upload file to VT 386a3e07209c3db65cebc66a0aab12596d260b181e5cba570eaaba0e080bda2b
2018-04-24 13:05:14 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-24 13:05:25 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-24 13:05:25 validate.py [line:141] DEBUG	success to upload file to VT 9fd52afe8dfb8058fa9567bf7b9ae1f62e82388e24786bce7a85570eb2b850e3
2018-04-24 13:05:25 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): ww

2018-04-24 13:08:41 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-24 13:08:41 validate.py [line:141] DEBUG	success to upload file to VT 99c1430bcbb8bf5c1b385281efa454c42653a0716034469d567b57120474acc1
2018-04-24 13:08:41 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-24 13:08:52 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-24 13:08:52 validate.py [line:141] DEBUG	success to upload file to VT e06521aa4264731bc59476dbf79ff8b7fd832e71554abb405f071949c3eaf790
2018-04-24 13:08:52 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-24 13:08:56 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST 

In [62]:
df = valid_obj.validate_file_with_vt(sha256_list)

2018-04-24 13:09:16 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-24 13:09:17 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff HTTP/1.1" 204 0
2018-04-24 13:09:17 validate.py [line:165] ERROR	get VT file report fail No JSON object could be decoded
2018-04-24 13:09:17 validate.py [line:168] DEBUG	index 0	 sha256 3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff	positives -1
2018-04-24 13:09:33 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-24 13:09:34 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c 

2018-04-24 13:13:55 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-24 13:14:04 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=c80800c753b09b05233d75e20abbf1173a4832f8e8a9daac9414d9243f2353aa HTTP/1.1" 200 None
2018-04-24 13:14:04 validate.py [line:168] DEBUG	index 15	 sha256 c80800c753b09b05233d75e20abbf1173a4832f8e8a9daac9414d9243f2353aa	positives 0
2018-04-24 13:14:20 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-24 13:14:24 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=18d5b7c0a71c44727342b2ce3b6692d72649ce6183e1e3300020f489c6d3304b HTTP/1.1" 200 None
2018-04-24 13:14:24 validate.py [line:168] DEBUG	index 16	 sha256 18d5b7c0a71c44727

2018-04-24 13:18:29 validate.py [line:168] DEBUG	index 30	 sha256 b3fd3381034a0e68416590c1cb47e2f5e1754d79edc0e0a45f1050f73a1e1511	positives 1
2018-04-24 13:18:45 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-24 13:18:47 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=2c11a9996f58ddb150ab4a8b5218c9e299c4481d4ac404fe3d73a1153bad1c81 HTTP/1.1" 200 None
2018-04-24 13:18:47 validate.py [line:168] DEBUG	index 31	 sha256 2c11a9996f58ddb150ab4a8b5218c9e299c4481d4ac404fe3d73a1153bad1c81	positives 0
2018-04-24 13:19:03 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-24 13:19:04 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=876445ab93a7a0076314611d

2018-04-24 13:23:09 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=30b81bf6142418faa7277cf6efd87c0912b6c25bbe25d3aea6c746586e29b5e9 HTTP/1.1" 200 None
2018-04-24 13:23:09 validate.py [line:168] DEBUG	index 46	 sha256 30b81bf6142418faa7277cf6efd87c0912b6c25bbe25d3aea6c746586e29b5e9	positives 0
2018-04-24 13:23:25 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-24 13:23:26 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=1b732de57fbb5e06cfd64e4952067f3d00a888ff10c7fdb89be4662193b0ec6b HTTP/1.1" 200 None
2018-04-24 13:23:26 validate.py [line:168] DEBUG	index 47	 sha256 1b732de57fbb5e06cfd64e4952067f3d00a888ff10c7fdb89be4662193b0ec6b	positives 15
2018-04-24 13:23:42 connectionpool.py [line:824]

2018-04-24 13:27:24 validate.py [line:168] DEBUG	index 61	 sha256 942904c1dd15b0cf86118326b9f077ab9d7ac0605a49f613213cee1fcba95bb8	positives 0
2018-04-24 13:27:40 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-24 13:27:41 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=28f33f961cf7f8f0a04b068e18ac3d556eeda0a251789c5705f4b0fc9ceade10 HTTP/1.1" 200 None
2018-04-24 13:27:41 validate.py [line:168] DEBUG	index 62	 sha256 28f33f961cf7f8f0a04b068e18ac3d556eeda0a251789c5705f4b0fc9ceade10	positives 0
2018-04-24 13:27:57 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-24 13:27:57 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=eace5bb394f4ba4362efbbd3

In [76]:
df.to_csv("../../data/EXP_domain/df_res_sha256.csv", index = False)